## Density Profiler Visualization:
1) Set up the parameters

2) Load and Plot interactive Graph

In [24]:
import seeq
from seeq import spy
import pandas

import numpy
import matplotlib

# import matplotlib.pyplot as plt
# import matplotlib.patches as mpatches

%matplotlib inline 
# %matplotlib notebook

# from ipywidgets import interactive, HBox, VBox, interact, Layout
import ipywidgets

import IPython

import datetime

In [18]:
userName = ipywidgets.Text(
    value='user@bp.com',
    placeholder='Type Username',
    description='Enter BP mail ',
    disabled=False,
style = {'description_width': 'initial'}
)
password = ipywidgets.Text(
    value='Password',
    placeholder='Type Password',
    description='Enter BP Password ',
    disabled=False,
style = {'description_width': 'initial'}
)

log = ipywidgets.HBox([userName,password])
IPython.display.display(log)

In [19]:
#Date Picker widgets
style = {'description_width': 'initial'}

startTime = ipywidgets.DatePicker(
    value = datetime.date.today() - datetime.timedelta(days=1),
    description='Starting date',
    disabled=False,
    style=style
)
startTimeh = ipywidgets.Text(
    value='01:00:00',
    placeholder='Type time',
    description='HH:MM:SS ',
    disabled=False
)

endTime = ipywidgets.DatePicker(
    value = datetime.date.today(),
    description='Ending date',
    disabled=False,
    style=style
)
endTimeh = ipywidgets.Text(
    value='00:00:00',
    placeholder='Type time',
    description='HH:MM:SS ',
    disabled=False
)

asset = ipywidgets.Dropdown(
    options=[('IP Sep', 'PSVM-1A-DI022**.PV PSVM-1A-XI02205**.PV'), ('Treater', 'PSVM-1A-DI023**.PV PSVM-1A-XI0230**.PV'),
             ('Desalter', 'PSVM-1A-DI024**.PV PSVM-1A-XI0240**.PV'), ('HP Sep A', 'PSVM-1A-DI02105**.PV PSVM-1A-XI02205**.PV')],
    value='PSVM-1A-DI022**.PV PSVM-1A-XI02205**.PV',
    description='Asset:',
)
fequency = ipywidgets.Dropdown(
    options=[('1Min', '1min'),('5Min','5min'),('15Min','15min'),('30Min','30min'),('60Min','60min')],
    value='1min',
    description='Frequency:',
)
v1 = ipywidgets.VBox([startTime,startTimeh])
v2 = ipywidgets.VBox([endTime,endTimeh])
v3 = ipywidgets.VBox([asset,fequency])
l = ipywidgets.HBox([v1,v2, v3])
IPython.display.display(l)

In [26]:
def getLogin():
    f = open("credentials.key", "w")   # 'r' for reading and 'w' for writing
    f.write("{} \n{}".format(userName.value,password.value))    # Write inside file 
    f.close()  
    
def getData(StartTime, EndTime, Freq, Sep, SepLimits):
    my_items = pandas.DataFrame({'Name': [Sep]})
    x = seeq.spy.search(my_items)
    my_data = seeq.spy.pull(x, start=StartTime, end=EndTime, grid=Freq, header='Name')
    my_data = my_data.reindex(sorted(my_data.columns), axis=1)
    # my_data

    my_new_items = pandas.DataFrame({'Name': [SepLimits]})
    x_limits = seeq.spy.search(my_new_items)

    my_data_limits = seeq.spy.pull(x_limits, start=StartTime, end=EndTime, grid=Freq, header='Name')
    # my_data = my_data.reindex(sorted(my_data.columns), axis=1)
    # my_data_limits
    return my_data, my_data_limits

def plotData(b):

    sep1 = asset.value.split()[0]
    sepLimits1 = asset.value.split()[1]
    startTime1 = startTime.value.strftime("%D") + " " + startTimeh.value
    endTime1 = endTime.value.strftime("%D") + " " + endTimeh.value
    freq = fequency.value#'60min'
    my_data, my_data_limits = getData(startTime1, endTime1, freq, sep1, sepLimits1)
    x = (numpy.arange(len(my_data.columns), 0, -1)/len(my_data.columns))*100

    IPython.display.clear_output(wait=True)
    slider = ipywidgets.IntSlider(value=1,min=1,max=len(my_data)-1,step=1,description='Date',layout={'width': '800px'})
    play = ipywidgets.Play(value=1,min=1,max=len(my_data)-1,step=1,interval=1211,description="Press play",disabled=False)
    g = ipywidgets.jslink((play, 'value'), (slider, 'value'))

    @ipywidgets.interact
    def plotF(a = slider,b=play):
        matplotlib.pyplot.rcParams['figure.figsize'] = (20,8)
        matplotlib.pyplot.rcParams.update({'font.size': 18})
        matplotlib.pyplot.rcParams['axes.xmargin'] = 0
        matplotlib.pyplot.rcParams['axes.ymargin'] = 0

        y = my_data.iloc[b]
        limits = my_data_limits.iloc[b].sort_values()

        Gas = y <= limits[0]
        Foam = (y > limits[0]) & (y <= limits[1])
        Oil = (y > limits[1]) & (y <= limits[2])
        Emul = (y > limits[2]) & (y <= limits[3])
        Water = (y > limits[3]) & (y <= limits[4])
        Sand = y > limits[4]

        x1 = matplotlib.pyplot.barh(x[Gas],y[Gas],color = 'gray', edgecolor='black', label='Gas')
        x2 = matplotlib.pyplot.barh(x[Foam],y[Foam],color = 'yellow', edgecolor='black', label='Foam')
        x3 = matplotlib.pyplot.barh(x[Oil],y[Oil],color = 'black', edgecolor='black', label='Oil')
        x4 = matplotlib.pyplot.barh(x[Emul],y[Emul],color = 'orange', edgecolor='black', label='Emul')
        x5 = matplotlib.pyplot.barh(x[Water],y[Water],color = 'blue', edgecolor='black', label='Water')
        x6 = matplotlib.pyplot.barh(x[Sand],y[Sand],color = 'red', edgecolor='black', label='Sand')


        gas = matplotlib.patches.Patch(color='gray', label='Gas')
        foam = matplotlib.patches.Patch(color='yellow', label='Foam')
        oil = matplotlib.patches.Patch(color='black', label='Oil')
        emul = matplotlib.patches.Patch(color='orange', label='Emul')
        water = matplotlib.patches.Patch(color='blue', label='Water')
        sand = matplotlib.patches.Patch(color='red', label='Sand')
        matplotlib.pyplot.legend(handles=[gas,foam,oil,emul,water,sand])
#         print(my_data.index[b].strftime("%b %d %Y %H:%M:%S"))
        matplotlib.pyplot.title('DP view for separator at: {}'.format(my_data.index[b].strftime("%b %d %Y %H:%M:%S")))
        matplotlib.pyplot.show()
    

        
# x,y = getData(startTime, endTime, freq, sep, sepLimits)
button = ipywidgets.Button(description="Load & Plot Data")

output = ipywidgets.Output()
IPython.display.display(button,output)


def on_button_clicked(b):

    with output:
        print("Button clicked.")
        getLogin()
        spy.login(url='https://seeq-eh.bpweb.bp.com', credentials_file = 'credentials.key', directory='Minimal AD configuration', ignore_ssl_errors=True)
        IPython.display.clear_output(wait=True)
        plotData(b)

button.on_click(on_button_clicked)



Button(description='Load & Plot Data', style=ButtonStyle())

Output()